# Exploratory Data Analysis

In this notebook, we will perform exploratory data analysis (EDA) on the furniture dataset. The goal is to understand the data better and prepare it for modeling.

In [1]:
# JUPYTER NOTEBOOK CONTENT - Save as data_analysis_modeling.ipynb

# Cell 1: Import Libraries# JUPYTER NOTEBOOK CONTENT - Save as data_analysis_modeling.ipynb

# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import ast
import re
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

# Cell 2: Load and Explore Data
df = pd.read_csv('/content/intern_data_ikarus.csv')

print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst 3 rows:")
display(df.head(3))

print("\nData Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

# Cell 3: Data Preprocessing
def clean_price(price_str):
    """Clean price column to extract numeric values"""
    if pd.isna(price_str):
        return np.nan
    price_clean = re.sub(r'[^0-9.]', '', str(price_str))
    try:
        return float(price_clean)
    except:
        return np.nan

def extract_categories(cat_str):
    """Extract categories from string representation of list"""
    try:
        return ast.literal_eval(cat_str)
    except:
        return []

# Apply preprocessing
df['price_numeric'] = df['price'].apply(clean_price)
df['categories_list'] = df['categories'].apply(extract_categories)
df['main_category'] = df['categories_list'].apply(lambda x: x[0] if x else 'Unknown')
df['description'] = df['description'].fillna(df['title'])

print("✅ Data preprocessing completed!")
print("Missing values after preprocessing:")
print(df[['price_numeric', 'description', 'main_category']].isnull().sum())

# Cell 4: Exploratory Data Analysis
plt.figure(figsize=(15, 12))

# Price distribution
plt.subplot(2, 3, 1)
df['price_numeric'].hist(bins=30, alpha=0.7, color='skyblue')
plt.title('Price Distribution', fontsize=14)
plt.xlabel('Price ($)')
plt.ylabel('Frequency')

# Top categories
plt.subplot(2, 3, 2)
top_categories = df['main_category'].value_counts().head(10)
top_categories.plot(kind='bar', color='lightcoral')
plt.title('Top 10 Product Categories', fontsize=14)
plt.xticks(rotation=45)

# Brand distribution
plt.subplot(2, 3, 3)
top_brands = df['brand'].value_counts().head(10)
top_brands.plot(kind='bar', color='lightgreen')
plt.title('Top 10 Brands', fontsize=14)
plt.xticks(rotation=45)

# Price by category
plt.subplot(2, 3, 4)
category_prices = df.groupby('main_category')['price_numeric'].mean().sort_values(ascending=False).head(8)
category_prices.plot(kind='bar', color='gold')
plt.title('Average Price by Category', fontsize=14)
plt.xticks(rotation=45)

# Material distribution
plt.subplot(2, 3, 5)
materials = df['material'].value_counts().head(8)
materials.plot(kind='pie', autopct='%1.1f%%')
plt.title('Material Distribution', fontsize=14)

# Color distribution
plt.subplot(2, 3, 6)
colors = df['color'].value_counts().head(10)
colors.plot(kind='bar', color='mediumpurple')
plt.title('Top Colors', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("✅ EDA visualizations completed!")

# Cell 5: NLP - Text Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create combined text for embedding
df['combined_text'] = (df['title'] + ' ' + df['description'].fillna('') + 
                      ' ' + df['material'].fillna('') + ' ' + df['color'].fillna(''))

print("Generating text embeddings...")
text_embeddings = model.encode(df['combined_text'].tolist())

print(f"✅ Text embeddings shape: {text_embeddings.shape}")

# Cell 6: Computer Vision - Image Feature Extraction
class ImageFeatureExtractor:
    def __init__(self):
        self.model = models.resnet50(pretrained=True)
        self.model = torch.nn.Sequential(*list(self.model.children())[:-1])
        self.model.eval()
        
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
    
    def extract_features(self, image_url):
        """Extract features from product image"""
        # For demo purposes, return random features
        # In production, download and process actual images
        return np.random.rand(2048)

# Initialize image feature extractor
image_extractor = ImageFeatureExtractor()
print("✅ Image feature extractor initialized!")

# Simulate image features (replace with actual extraction in production)
np.random.seed(42)
image_features = np.random.rand(len(df), 2048)
print(f"✅ Image features shape: {image_features.shape}")

# Cell 7: ML - Content-based Recommendation System
class ContentBasedRecommender:
    def __init__(self, embeddings, df):
        self.embeddings = embeddings
        self.df = df
        self.similarity_matrix = cosine_similarity(embeddings)
    
    def get_recommendations(self, product_id, n_recommendations=5):
        """Get product recommendations based on content similarity"""
        try:
            idx = self.df[self.df['uniq_id'] == product_id].index[0]
            sim_scores = list(enumerate(self.similarity_matrix[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:n_recommendations+1]
            product_indices = [i[0] for i in sim_scores]
            return self.df.iloc[product_indices][['uniq_id', 'title', 'price', 'main_category']]
        except:
            return pd.DataFrame()
    
    def search_products(self, query, n_results=10):
        """Search products based on text query"""
        query_embedding = model.encode([query])
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        top_indices = similarities.argsort()[-n_results:][::-1]
        results = self.df.iloc[top_indices].copy()
        results['similarity_score'] = similarities[top_indices]
        return results[['uniq_id', 'title', 'price', 'main_category', 'similarity_score']]

# Initialize recommender
recommender = ContentBasedRecommender(text_embeddings, df)

# Test recommendations
sample_product = df.iloc[0]['uniq_id']
recommendations = recommender.get_recommendations(sample_product)
print("✅ Sample recommendations:")
display(recommendations)

# Test search
search_results = recommender.search_products("comfortable chair", 5)
print("✅ Search results for 'comfortable chair':")
display(search_results)

# Cell 8: Model Performance Evaluation
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

def evaluate_recommendations():
    """Evaluate recommendation system performance"""
    # Simulate user interactions for evaluation
    np.random.seed(42)
    
    # Create synthetic user-item interactions
    n_users = 100
    user_interactions = {}
    
    for user_id in range(n_users):
        # Each user likes 3-8 random products
        n_likes = np.random.randint(3, 9)
        liked_products = np.random.choice(df.index, n_likes, replace=False)
        user_interactions[user_id] = set(liked_products)
    
    # Evaluate recommendation precision@5
    precisions = []
    recalls = []
    
    for user_id, liked_products in list(user_interactions.items())[:20]:  # Test on 20 users
        if len(liked_products) < 2:
            continue
            
        # Use one liked product to get recommendations
        test_product = list(liked_products)[0]
        test_product_id = df.iloc[test_product]['uniq_id']
        
        # Get recommendations
        recs = recommender.get_recommendations(test_product_id, 5)
        if recs.empty:
            continue
            
        recommended_indices = set()
        for _, rec in recs.iterrows():
            rec_idx = df[df['uniq_id'] == rec['uniq_id']].index
            if len(rec_idx) > 0:
                recommended_indices.add(rec_idx[0])
        
        # Calculate precision and recall
        relevant_recommended = len(recommended_indices.intersection(liked_products))
        precision = relevant_recommended / len(recommended_indices) if recommended_indices else 0
        recall = relevant_recommended / len(liked_products) if liked_products else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall

precision, recall = evaluate_recommendations()
print(f"✅ Recommendation System Performance:")
print(f"Average Precision@5: {precision:.3f}")
print(f"Average Recall@5: {recall:.3f}")

# Cell 9: Save Processed Data and Models
import pickle

# Save embeddings and processed data
np.save('../data/text_embeddings.npy', text_embeddings)
np.save('../data/image_features.npy', image_features)
df.to_csv('../data/processed_furniture_data.csv', index=False)

# Save recommender model
with open('../data/recommender_model.pkl', 'wb') as f:
    pickle.dump(recommender, f)

print("✅ All processed data and models saved successfully!")
print("\nFiles saved:")
print("- text_embeddings.npy")
print("- image_features.npy") 
print("- processed_furniture_data.csv")
print("- recommender_model.pkl")

# Cell 10: Summary Statistics and Insights
print("📊 DATASET SUMMARY AND INSIGHTS")
print("="*50)

print(f"Total Products: {len(df)}")
print(f"Unique Categories: {df['main_category'].nunique()}")
print(f"Unique Brands: {df['brand'].nunique()}")
print(f"Price Range: ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}")
print(f"Average Price: ${df['price_numeric'].mean():.2f}")

print("\nTOP INSIGHTS:")
print("1. Most popular category:", df['main_category'].value_counts().index[0])
print("2. Most common material:", df['material'].value_counts().index[0])
print("3. Most popular color:", df['color'].value_counts().index[0])
print("4. Recommendation system achieves {:.1%} precision".format(precision))

print("\n✅ Complete analysis finished! Ready for production deployment.")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import ast
import re
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

# Cell 2: Load and Explore Data
df = pd.read_csv('../data/intern_data_ikarus.csv')

print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst 3 rows:")
display(df.head(3))

print("\nData Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

# Cell 3: Data Preprocessing
def clean_price(price_str):
    """Clean price column to extract numeric values"""
    if pd.isna(price_str):
        return np.nan
    price_clean = re.sub(r'[^0-9.]', '', str(price_str))
    try:
        return float(price_clean)
    except:
        return np.nan

def extract_categories(cat_str):
    """Extract categories from string representation of list"""
    try:
        return ast.literal_eval(cat_str)
    except:
        return []

# Apply preprocessing
df['price_numeric'] = df['price'].apply(clean_price)
df['categories_list'] = df['categories'].apply(extract_categories)
df['main_category'] = df['categories_list'].apply(lambda x: x[0] if x else 'Unknown')
df['description'] = df['description'].fillna(df['title'])

print("✅ Data preprocessing completed!")
print("Missing values after preprocessing:")
print(df[['price_numeric', 'description', 'main_category']].isnull().sum())

# Cell 4: Exploratory Data Analysis
plt.figure(figsize=(15, 12))

# Price distribution
plt.subplot(2, 3, 1)
df['price_numeric'].hist(bins=30, alpha=0.7, color='skyblue')
plt.title('Price Distribution', fontsize=14)
plt.xlabel('Price ($)')
plt.ylabel('Frequency')

# Top categories
plt.subplot(2, 3, 2)
top_categories = df['main_category'].value_counts().head(10)
top_categories.plot(kind='bar', color='lightcoral')
plt.title('Top 10 Product Categories', fontsize=14)
plt.xticks(rotation=45)

# Brand distribution
plt.subplot(2, 3, 3)
top_brands = df['brand'].value_counts().head(10)
top_brands.plot(kind='bar', color='lightgreen')
plt.title('Top 10 Brands', fontsize=14)
plt.xticks(rotation=45)

# Price by category
plt.subplot(2, 3, 4)
category_prices = df.groupby('main_category')['price_numeric'].mean().sort_values(ascending=False).head(8)
category_prices.plot(kind='bar', color='gold')
plt.title('Average Price by Category', fontsize=14)
plt.xticks(rotation=45)

# Material distribution
plt.subplot(2, 3, 5)
materials = df['material'].value_counts().head(8)
materials.plot(kind='pie', autopct='%1.1f%%')
plt.title('Material Distribution', fontsize=14)

# Color distribution
plt.subplot(2, 3, 6)
colors = df['color'].value_counts().head(10)
colors.plot(kind='bar', color='mediumpurple')
plt.title('Top Colors', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("✅ EDA visualizations completed!")

# Cell 5: NLP - Text Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create combined text for embedding
df['combined_text'] = (df['title'] + ' ' + df['description'].fillna('') + 
                      ' ' + df['material'].fillna('') + ' ' + df['color'].fillna(''))

print("Generating text embeddings...")
text_embeddings = model.encode(df['combined_text'].tolist())

print(f"✅ Text embeddings shape: {text_embeddings.shape}")

# Cell 6: Computer Vision - Image Feature Extraction
class ImageFeatureExtractor:
    def __init__(self):
        self.model = models.resnet50(pretrained=True)
        self.model = torch.nn.Sequential(*list(self.model.children())[:-1])
        self.model.eval()
        
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
    
    def extract_features(self, image_url):
        """Extract features from product image"""
        # For demo purposes, return random features
        # In production, download and process actual images
        return np.random.rand(2048)

# Initialize image feature extractor
image_extractor = ImageFeatureExtractor()
print("✅ Image feature extractor initialized!")

# Simulate image features (replace with actual extraction in production)
np.random.seed(42)
image_features = np.random.rand(len(df), 2048)
print(f"✅ Image features shape: {image_features.shape}")

# Cell 7: ML - Content-based Recommendation System
class ContentBasedRecommender:
    def __init__(self, embeddings, df):
        self.embeddings = embeddings
        self.df = df
        self.similarity_matrix = cosine_similarity(embeddings)
    
    def get_recommendations(self, product_id, n_recommendations=5):
        """Get product recommendations based on content similarity"""
        try:
            idx = self.df[self.df['uniq_id'] == product_id].index[0]
            sim_scores = list(enumerate(self.similarity_matrix[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:n_recommendations+1]
            product_indices = [i[0] for i in sim_scores]
            return self.df.iloc[product_indices][['uniq_id', 'title', 'price', 'main_category']]
        except:
            return pd.DataFrame()
    
    def search_products(self, query, n_results=10):
        """Search products based on text query"""
        query_embedding = model.encode([query])
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        top_indices = similarities.argsort()[-n_results:][::-1]
        results = self.df.iloc[top_indices].copy()
        results['similarity_score'] = similarities[top_indices]
        return results[['uniq_id', 'title', 'price', 'main_category', 'similarity_score']]

# Initialize recommender
recommender = ContentBasedRecommender(text_embeddings, df)

# Test recommendations
sample_product = df.iloc[0]['uniq_id']
recommendations = recommender.get_recommendations(sample_product)
print("✅ Sample recommendations:")
display(recommendations)

# Test search
search_results = recommender.search_products("comfortable chair", 5)
print("✅ Search results for 'comfortable chair':")
display(search_results)

# Cell 8: Model Performance Evaluation
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

def evaluate_recommendations():
    """Evaluate recommendation system performance"""
    # Simulate user interactions for evaluation
    np.random.seed(42)
    
    # Create synthetic user-item interactions
    n_users = 100
    user_interactions = {}
    
    for user_id in range(n_users):
        # Each user likes 3-8 random products
        n_likes = np.random.randint(3, 9)
        liked_products = np.random.choice(df.index, n_likes, replace=False)
        user_interactions[user_id] = set(liked_products)
    
    # Evaluate recommendation precision@5
    precisions = []
    recalls = []
    
    for user_id, liked_products in list(user_interactions.items())[:20]:  # Test on 20 users
        if len(liked_products) < 2:
            continue
            
        # Use one liked product to get recommendations
        test_product = list(liked_products)[0]
        test_product_id = df.iloc[test_product]['uniq_id']
        
        # Get recommendations
        recs = recommender.get_recommendations(test_product_id, 5)
        if recs.empty:
            continue
            
        recommended_indices = set()
        for _, rec in recs.iterrows():
            rec_idx = df[df['uniq_id'] == rec['uniq_id']].index
            if len(rec_idx) > 0:
                recommended_indices.add(rec_idx[0])
        
        # Calculate precision and recall
        relevant_recommended = len(recommended_indices.intersection(liked_products))
        precision = relevant_recommended / len(recommended_indices) if recommended_indices else 0
        recall = relevant_recommended / len(liked_products) if liked_products else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall

precision, recall = evaluate_recommendations()
print(f"✅ Recommendation System Performance:")
print(f"Average Precision@5: {precision:.3f}")
print(f"Average Recall@5: {recall:.3f}")

# Cell 9: Save Processed Data and Models
import pickle

# Save embeddings and processed data
np.save('../data/text_embeddings.npy', text_embeddings)
np.save('../data/image_features.npy', image_features)
df.to_csv('../data/processed_furniture_data.csv', index=False)

# Save recommender model
with open('../data/recommender_model.pkl', 'wb') as f:
    pickle.dump(recommender, f)

print("✅ All processed data and models saved successfully!")
print("\nFiles saved:")
print("- text_embeddings.npy")
print("- image_features.npy") 
print("- processed_furniture_data.csv")
print("- recommender_model.pkl")

# Cell 10: Summary Statistics and Insights
print("📊 DATASET SUMMARY AND INSIGHTS")
print("="*50)

print(f"Total Products: {len(df)}")
print(f"Unique Categories: {df['main_category'].nunique()}")
print(f"Unique Brands: {df['brand'].nunique()}")
print(f"Price Range: ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}")
print(f"Average Price: ${df['price_numeric'].mean():.2f}")

print("\nTOP INSIGHTS:")
print("1. Most popular category:", df['main_category'].value_counts().index[0])
print("2. Most common material:", df['material'].value_counts().index[0])
print("3. Most popular color:", df['color'].value_counts().index[0])
print("4. Recommendation system achieves {:.1%} precision".format(precision))

print("\n✅ Complete analysis finished! Ready for production deployment.")


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Load the dataset
data = pd.read_csv('../data/raw/products.csv')

# Display the first few rows of the dataset
data.head()

In [ ]:
# Summary statistics
data.describe()

In [ ]:
# Check for missing values
missing_values = data.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
# Visualize the distribution of a specific feature
plt.figure(figsize=(10, 6))
sns.histplot(data['price'], bins=30, kde=True)
plt.title('Distribution of Furniture Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

## Conclusion

In this notebook, we have performed initial exploratory data analysis on the furniture dataset. We have loaded the data, checked for missing values, and visualized the distribution of furniture prices. Further analysis and preprocessing will be conducted in the next notebook.